In [23]:
import pandas as pd
df1 = pd.read_csv('./data/국립공원공단_국립공원 로드킬 정보_20231220.csv', encoding='cp949')
df1.head()

,국립공원명,조사일자,자원명,경도,위도
0,한려해상,2021-10-20,멧새,128.674692,34.752264
1,한려해상,2021-10-13,청설모,128.674692,34.752264
2,한려해상,2021-10-11,청설모,128.674692,34.752264
3,한려해상,2021-09-02,능구렁이,128.674692,34.752264
4,한려해상,2021-07-27,고라니,128.674692,34.752264


In [24]:
df1['조사일자'] = pd.to_datetime(df1['조사일자'])

In [25]:
# Count 계산
count_df = df1.groupby(['국립공원명', df1['조사일자'].dt.year, '경도', '위도']).size().reset_index(name='Count')

# 새로운 데이터프레임 생성
df1 = count_df[['국립공원명', '조사일자', '경도', '위도', 'Count']]
df1.columns = ['국립공원명', 'Year', '경도', '위도', 'Count']

# 결과 출력
df1

,국립공원명,Year,경도,위도,Count
0,가야산,2011,128.099910,35.807853,3
1,가야산,2012,128.099910,35.807853,5
2,가야산,2012,128.113948,35.783700,2
3,가야산,2013,128.099910,35.807853,4
4,가야산,2014,128.099910,35.807853,1
...,...,...,...,...,...
556,한려해상,2022,128.530303,34.760100,12
557,한려해상,2022,128.614994,34.715500,1
558,한려해상,2022,128.651074,34.770900,2
559,한려해상,2022,128.674692,34.752264,7


In [26]:
import pandas as pd

df2 = pd.read_csv('./data/도로공사 로드킬 (2019-2022).csv')
df2.head()

,본부명,지사명,노선명,방향,발생건수,위도,경도,Year
0,수도권,경기광주,제2중부선,하남,4.0,37.283,127.386,2019
1,수도권,경기광주,중부선,통영,7.0,37.249,127.421,2019
2,수도권,경기광주,중부선,통영,10.0,37.283,127.385,2019
3,수도권,경기광주,중부선,통영,7.0,37.315,127.347,2019
4,수도권,경기광주,중부선,통영,4.0,37.354,127.321,2019


In [45]:
# 두 데이터프레임을 'Year', '경도', '위도'를 기준으로 외부 조인
df = pd.merge(df1, df2, on=['Year', '경도', '위도'], how='outer')

# '발생건수' 열을 최댓값으로 설정
df['발생건수'] = df[['Count', '발생건수']].max(axis=1)

# 불필요한 열 제거
df.drop(columns=['Count'], inplace=True)

# 결과 출력
df

,국립공원명,Year,경도,위도,본부명,지사명,노선명,방향,발생건수
0,태안해안,2011,126.328775,36.554600,NaN,NaN,NaN,NaN,1.0
1,변산반도,2011,126.582075,35.647022,NaN,NaN,NaN,NaN,19.0
2,변산반도,2011,126.597100,35.659800,NaN,NaN,NaN,NaN,8.0
3,내장산,2011,126.846113,35.447600,NaN,NaN,NaN,NaN,14.0
4,내장산,2011,126.855392,35.470908,NaN,NaN,NaN,NaN,9.0
...,...,...,...,...,...,...,...,...,...
1040,소백산,2023,128.454856,36.900700,NaN,NaN,NaN,NaN,1.0
1041,설악산,2023,128.529913,38.179000,NaN,NaN,NaN,NaN,1.0
1042,오대산,2023,128.580982,37.755300,NaN,NaN,NaN,NaN,9.0
1043,오대산,2023,128.616946,37.768800,NaN,NaN,NaN,NaN,15.0


In [46]:
df.to_csv('./data/roadkill.csv')

In [47]:
import folium
from folium.plugins import HeatMap

korea = folium.Map(location=[36.55, 127.75], zoom_start=7)
HeatMap(data=df[['위도', '경도']], radius=10).add_to(korea)

korea

In [50]:
import folium

# 작은 아이콘 이미지 경로
icon_url_red = 'https://raw.githubusercontent.com/pointhi/leaflet-color-markers/master/img/marker-icon-2x-red.png'

# 지도의 초기 위치와 확대 수준 설정
korea = folium.Map(location=[36.55, 127.75], zoom_start=7)

# df 데이터프레임에 있는 각 위치에 작은 빨간색 마커 추가
for idx, row in df.iterrows():
    icon = folium.CustomIcon(
        icon_url_red,
        icon_size=(10, 10)  # 아이콘의 크기 설정
    )
    marker = folium.Marker(
        location=[row['위도'], row['경도']],
        icon=icon
    )
    marker.add_to(korea)

# 지도 표시
korea

In [48]:
df.columns

Index(['국립공원명', 'Year', '경도', '위도', '본부명', '지사명', '노선명', '방향', '발생건수'], dtype='object')

In [49]:
import folium
from folium.plugins import MarkerCluster

# 지도의 초기 위치와 확대 수준 설정
korea = folium.Map(location=[36.55, 127.75], zoom_start=7)

# 마커 클러스터 생성
mc = MarkerCluster()

# df 데이터프레임의 각 행에 대해 마커 추가
for _, row in df.iterrows():
    # NaN 값이 있는 경우 빈 문자열로 대체하여 문자열로 변환
    park_name = row['국립공원명'] if pd.notnull(row['국립공원명']) else ''
    head_office = row['본부명'] if pd.notnull(row['본부명']) else ''
    branch_office = row['지사명'] if pd.notnull(row['지사명']) else ''
    route_name = row['노선명'] if pd.notnull(row['노선명']) else ''
    
    folium.Marker(
        location=[row['위도'], row['경도']],
        popup=park_name + '\n' + head_office + '\n' + branch_office + '\n' + route_name, 
        icon=folium.Icon(color='red')  # 빨간색 마커
    ).add_to(mc)

# 마커 클러스터를 지도에 추가
korea.add_child(mc)

# 지도 표시
korea

